In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
REVERSE = True
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128

### Independent Variables 
DATA_SIZE = 30000
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+ '
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)
print(ctable.chars)
print(ctable.char_indices)
print(ctable.indices_char)

[' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
{0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}


# Data Generation

In [6]:
def FormulaGenerate(ValueNum=2, opNum=1, digits=4, operator='+-'):
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, digits + 1))))
    fop = lambda: ''.join(np.random.choice(list(operator)) for i in range(1))
    values = []
    ops = []
    for i in range(ValueNum):
        values.append(f())
    for i in range(opNum):
        ops.append(fop())
    return values, ops

def DataGenerate(operator='+-', maxlen=7, digits=3, ansmaxlen=4, combime=False):
    questions = []
    shows = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        if combime == False:
            values, ops = FormulaGenerate(ValueNum=2, opNum=1, digits=digits, operator=operator)
            a, b, c = values[0], values[1], ops[0]
            a, b = sorted((a, b))[::-1]
            key = tuple((a, b, c))
            if key in seen:
                continue
            seen.add(key)
            q = '{}{}{}'.format(a, c, b)
            
            if c == '-':
                ans = str(a - b)
            elif c == '+':
                ans = str(a + b)
            else:
                ans = str(a * b)
        else:
            while(1):
                values, ops = FormulaGenerate(ValueNum=3, opNum=2, digits=digits, operator=operator)
                a, b, c, d, e = values[0], values[1], values[2], ops[0], ops[1]
                a, b, c = sorted((a, b, c))[::-1]
                if d == '-' and e == '-':
                    ans =  a - b - c
                elif d == '-':
                    ans =  a - b + c
                elif e == '-':
                    ans =  a + b - c
                else:
                    ans = a + b + c
                if ans >= 0:
                    ans = str(ans)
                    break
            key = tuple((a, b, c))
            if key in seen:
                continue
            seen.add(key)
            q = '{}{}{}{}{}'.format(a, d, b, e, c)
        query = q + ' ' * (maxlen - len(q))
        ans += ' ' * (ansmaxlen - len(ans))
        if REVERSE:
            show = query
            query = query[::-1]
        questions.append(query)
        shows.append(show)
        expected.append(ans)
    print('Total addition questions:', len(questions))
    return questions, expected, shows

In [7]:
questions, expected, shows = DataGenerate('+', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
print(shows[:5], expected[:5])

Generating data...
Total addition questions: 30000
['498+43 ', '177+7  ', '68+2   ', '36+1   ', '78+2   '] ['541 ', '184 ', '70  ', '37  ', '80  ']


# Processing

In [8]:
def Vectorization(questions, expected, ctable, maxlen=7, ansmaxlen=4):
    print('Vectorization...')
    x = np.zeros((len(questions), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(expected), ansmaxlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, maxlen)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, ansmaxlen)
    return x, y
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=DIGITS+1)

Vectorization...


In [9]:
def shuffle(x, y, ratio=0.25):
    indices = np.arange(len(y))
    #np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]
    
    split = int(len(y) * ratio)
    # train_test_split
    train_x = x[split:]
    train_y = y[split:]
    test_x = x[:split]
    test_y = y[:split]

    split_at = len(train_x) - len(train_x) // 10
    (x_train, x_val) = train_x[:split_at], train_x[split_at:]
    (y_train, y_val) = train_y[:split_at], train_y[split_at:]
    return x_train, y_train, x_val, y_val, test_x, test_y

x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

Training Data:
(20250, 7, 12)
(20250, 4, 12)
Validation Data:
(2250, 7, 12)
(2250, 4, 12)
Testing Data:
(7500, 7, 12)
(7500, 4, 12)
input:  [[[ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]]

 [[ True False False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False

# Build Model

In [10]:
def BuildModel(maxlen=7, ansmaxlen=4):
    print('Build model...')

    RNN = layers.LSTM
    HIDDEN_SIZE = 128 * ansmaxlen
    BATCH_SIZE = 128
    LAYERS = 1

    model = Sequential()
    model.add(RNN(HIDDEN_SIZE))
    model.add(layers.Reshape((ansmaxlen, 128)))
    for _ in range(LAYERS):
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.Dense(len(chars), activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

#     model.build()
#     model.summary()
    return model

# Training part

In [11]:
model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Build model...

--------------------------------------------------
Iteration 0
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 9s 428us/step - loss: 1.8857 - acc: 0.3258 - val_loss: 1.7803 - val_acc: 0.3469
Q 687+207 T 894  ☒ 102 
Q 522+46  T 568  ☒ 402 
Q 593+21  T 614  ☒ 402 
Q 293+23  T 316  ☒ 402 
Q 619+52  T 671  ☒ 402 
Q 282+42  T 324  ☒ 402 
Q 42+38   T 80   ☒ 402 
Q 487+6   T 493  ☒ 409 
Q 851+86  T 937  ☒ 102 
Q 628+353 T 981  ☒ 102 

--------------------------------------------------
Iteration 1
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 221us/step - loss: 1.6178 - acc: 0.3896 - val_loss: 1.5743 - val_acc: 0.3984
Q 283+51  T 334  ☒ 299 
Q 180+73  T 253  ☒ 109 
Q 598+6   T 604  ☒ 599 
Q 960+5   T 965  ☒ 109 
Q 737+382 T 1119 ☒ 1313
Q 294+85  T 379  ☒ 299 
Q 472+67  T 539  ☒ 599 
Q 883+241 T 1124 ☒ 1593
Q 965+69  T 1034 ☒ 1093
Q 815+69  T 884  ☒ 109 

----

20250/20250 [==============================] - 4s 210us/step - loss: 0.0875 - acc: 0.9718 - val_loss: 0.0573 - val_acc: 0.9822
Q 189+55  T 244  ☑ 244 
Q 687+8   T 695  ☑ 695 
Q 201+141 T 342  ☑ 342 
Q 879+607 T 1486 ☑ 1486
Q 373+248 T 621  ☑ 621 
Q 855+5   T 860  ☑ 860 
Q 595+362 T 957  ☑ 957 
Q 659+91  T 750  ☑ 750 
Q 457+197 T 654  ☒ 653 
Q 80+49   T 129  ☑ 129 

--------------------------------------------------
Iteration 15
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 209us/step - loss: 0.0305 - acc: 0.9920 - val_loss: 0.0463 - val_acc: 0.9849
Q 766+66  T 832  ☑ 832 
Q 203+65  T 268  ☑ 268 
Q 819+31  T 850  ☑ 850 
Q 216+4   T 220  ☑ 220 
Q 153+63  T 216  ☑ 216 
Q 427+73  T 500  ☑ 500 
Q 644+319 T 963  ☑ 963 
Q 139+3   T 142  ☑ 142 
Q 369+68  T 437  ☑ 437 
Q 166+53  T 219  ☑ 219 

--------------------------------------------------
Iteration 16
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 209us/step - loss: 0.0016 - acc: 0.9999 - val_loss: 0.0102 - val_acc: 0.9962
Q 629+35  T 664  ☑ 664 
Q 998+59  T 1057 ☑ 1057
Q 570+27  T 597  ☑ 597 
Q 845+354 T 1199 ☑ 1199
Q 998+257 T 1255 ☑ 1255
Q 95+63   T 158  ☑ 158 
Q 872+357 T 1229 ☑ 1229
Q 893+601 T 1494 ☑ 1494
Q 751+97  T 848  ☑ 848 
Q 865+66  T 931  ☑ 931 

--------------------------------------------------
Iteration 29
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 211us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0536 - val_acc: 0.9831
Q 166+20  T 186  ☑ 186 
Q 32+19   T 51   ☑ 51  
Q 528+8   T 536  ☑ 536 
Q 595+362 T 957  ☑ 957 
Q 599+51  T 650  ☑ 650 
Q 626+32  T 658  ☑ 658 
Q 999+702 T 1701 ☒ 1601
Q 813+1   T 814  ☑ 814 
Q 23+21   T 44   ☑ 44  
Q 255+33  T 288  ☑ 288 

--------------------------------------------------
Iteration 30
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 211us/step - loss: 5.5428e-04 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 0.9973
Q 533+518 T 1051 ☑ 1051
Q 865+682 T 1547 ☑ 1547
Q 794+91  T 885  ☑ 885 
Q 345+16  T 361  ☑ 361 
Q 730+2   T 732  ☑ 732 
Q 717+276 T 993  ☑ 993 
Q 779+99  T 878  ☑ 878 
Q 628+7   T 635  ☑ 635 
Q 971+49  T 1020 ☑ 1020
Q 521+61  T 582  ☑ 582 

--------------------------------------------------
Iteration 43
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 210us/step - loss: 0.0999 - acc: 0.9690 - val_loss: 0.0418 - val_acc: 0.9872
Q 432+1   T 433  ☑ 433 
Q 121+20  T 141  ☑ 141 
Q 549+43  T 592  ☑ 592 
Q 442+94  T 536  ☑ 536 
Q 532+76  T 608  ☑ 608 
Q 495+76  T 571  ☑ 571 
Q 858+57  T 915  ☑ 915 
Q 971+41  T 1012 ☑ 1012
Q 243+71  T 314  ☑ 314 
Q 559+2   T 561  ☑ 561 

--------------------------------------------------
Iteration 44
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=

20250/20250 [==============================] - 4s 210us/step - loss: 0.0324 - acc: 0.9898 - val_loss: 0.0286 - val_acc: 0.9903
Q 906+44  T 950  ☑ 950 
Q 738+673 T 1411 ☑ 1411
Q 973+9   T 982  ☑ 982 
Q 339+27  T 366  ☑ 366 
Q 973+9   T 982  ☑ 982 
Q 858+50  T 908  ☑ 908 
Q 337+85  T 422  ☑ 422 
Q 489+252 T 741  ☑ 741 
Q 898+617 T 1515 ☒ 1516
Q 290+12  T 302  ☑ 302 

--------------------------------------------------
Iteration 57
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 209us/step - loss: 0.0057 - acc: 0.9988 - val_loss: 0.0101 - val_acc: 0.9968
Q 544+36  T 580  ☑ 580 
Q 29+25   T 54   ☑ 54  
Q 828+22  T 850  ☑ 850 
Q 707+53  T 760  ☑ 760 
Q 877+3   T 880  ☑ 880 
Q 598+55  T 653  ☑ 653 
Q 229+1   T 230  ☑ 230 
Q 262+83  T 345  ☑ 345 
Q 986+57  T 1043 ☑ 1043
Q 909+80  T 989  ☑ 989 

--------------------------------------------------
Iteration 58
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 212us/step - loss: 1.5058e-04 - acc: 1.0000 - val_loss: 0.0057 - val_acc: 0.9980
Q 517+221 T 738  ☑ 738 
Q 247+11  T 258  ☑ 258 
Q 549+63  T 612  ☑ 612 
Q 550+202 T 752  ☑ 752 
Q 737+91  T 828  ☑ 828 
Q 517+45  T 562  ☑ 562 
Q 404+5   T 409  ☑ 409 
Q 111+84  T 195  ☑ 195 
Q 821+5   T 826  ☑ 826 
Q 871+477 T 1348 ☑ 1348

--------------------------------------------------
Iteration 71
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 210us/step - loss: 1.3577e-04 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 0.9979
Q 725+73  T 798  ☑ 798 
Q 806+145 T 951  ☑ 951 
Q 588+85  T 673  ☑ 673 
Q 850+36  T 886  ☑ 886 
Q 244+2   T 246  ☑ 246 
Q 130+32  T 162  ☑ 162 
Q 588+85  T 673  ☑ 673 
Q 647+6   T 653  ☑ 653 
Q 510+127 T 637  ☑ 637 
Q 367+11  T 378  ☑ 378 

--------------------------------------------------
Iteration 72
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/2025

20250/20250 [==============================] - 4s 211us/step - loss: 4.1588e-05 - acc: 1.0000 - val_loss: 0.0060 - val_acc: 0.9977
Q 459+134 T 593  ☑ 593 
Q 998+675 T 1673 ☑ 1673
Q 794+91  T 885  ☑ 885 
Q 86+33   T 119  ☑ 119 
Q 973+5   T 978  ☑ 978 
Q 863+19  T 882  ☑ 882 
Q 290+178 T 468  ☑ 468 
Q 508+15  T 523  ☑ 523 
Q 957+557 T 1514 ☑ 1514
Q 942+48  T 990  ☑ 990 

--------------------------------------------------
Iteration 85
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 210us/step - loss: 0.1070 - acc: 0.9692 - val_loss: 0.0351 - val_acc: 0.9894
Q 271+4   T 275  ☑ 275 
Q 475+72  T 547  ☑ 547 
Q 248+31  T 279  ☑ 279 
Q 971+77  T 1048 ☑ 1048
Q 145+22  T 167  ☑ 167 
Q 235+0   T 235  ☑ 235 
Q 627+58  T 685  ☑ 685 
Q 812+17  T 829  ☑ 829 
Q 900+85  T 985  ☑ 985 
Q 459+22  T 481  ☑ 481 

--------------------------------------------------
Iteration 86
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=

20250/20250 [==============================] - 4s 210us/step - loss: 1.4892e-04 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 0.9978
Q 84+73   T 157  ☑ 157 
Q 859+0   T 859  ☑ 859 
Q 628+447 T 1075 ☑ 1075
Q 566+53  T 619  ☑ 619 
Q 427+5   T 432  ☑ 432 
Q 963+9   T 972  ☑ 972 
Q 325+110 T 435  ☑ 435 
Q 582+529 T 1111 ☑ 1111
Q 354+132 T 486  ☑ 486 
Q 11+11   T 22   ☑ 22  

--------------------------------------------------
Iteration 99
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 210us/step - loss: 1.3377e-04 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9978
Q 876+609 T 1485 ☑ 1485
Q 710+190 T 900  ☑ 900 
Q 558+242 T 800  ☒ 700 
Q 603+96  T 699  ☑ 699 
Q 93+82   T 175  ☑ 175 
Q 562+90  T 652  ☑ 652 
Q 967+516 T 1483 ☑ 1483
Q 386+67  T 453  ☑ 453 
Q 469+356 T 825  ☑ 825 
Q 387+92  T 479  ☑ 479 


### Analysis
* Only addition
* 3 digits
* Training size: 20.25K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 16 epoch

In [12]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+ '
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 347us/step - loss: 1.7302 - acc: 0.3651 - val_loss: 1.5169 - val_acc: 0.4499
Q 2007+1124 T 3131  ☒ 2101 
Q 2482+9    T 2491  ☒ 2281 
Q 650+8     T 658   ☒ 601  
Q 848+100   T 948   ☒ 144  
Q 675+279   T 954   ☒ 700  
Q 6107+359  T 6466  ☒ 7008 
Q 562+282   T 844   ☒ 620  
Q 647+7     T 654   ☒ 641  
Q 2507+1    T 2508  ☒ 2281 
Q 3228+20   T 3248  ☒ 3281 

--------------------------------------------------
Iteration 1
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 1.2219 - acc: 0.5310 - val_loss: 1.0693 - val_acc: 0.5961
Q 7396+876  T 8272  ☒ 8074 
Q 806+551   T 1357  ☒ 1350 
Q 5825+1960 T 7785  ☒ 8074 
Q 51+16     T 67    ☒ 70   
Q 9220+3    T 9223  ☒ 9

40500/40500 [==============================] - 11s 278us/step - loss: 0.0304 - acc: 0.9911 - val_loss: 0.0295 - val_acc: 0.9894
Q 2647+7    T 2654  ☑ 2654 
Q 918+373   T 1291  ☑ 1291 
Q 9009+72   T 9081  ☑ 9081 
Q 65+39     T 104   ☑ 104  
Q 7670+7    T 7677  ☑ 7677 
Q 3772+64   T 3836  ☑ 3836 
Q 3772+64   T 3836  ☑ 3836 
Q 7126+5833 T 12959 ☒ 12969
Q 8436+767  T 9203  ☑ 9203 
Q 3423+32   T 3455  ☑ 3455 

--------------------------------------------------
Iteration 14
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0205 - acc: 0.9944 - val_loss: 0.0901 - val_acc: 0.9700
Q 365+3     T 368   ☑ 368  
Q 7632+9    T 7641  ☑ 7641 
Q 1396+7    T 1403  ☑ 1403 
Q 8930+609  T 9539  ☒ 9549 
Q 3880+3    T 3883  ☑ 3883 
Q 3065+6    T 3071  ☑ 3071 
Q 454+60    T 514   ☑ 514  
Q 4134+215  T 4349  ☑ 4349 
Q 2484+919  T 3403  ☑ 3403 
Q 975+82    T 1057  ☑ 1057 

--------------------------------------------------
Iteratio

Q 836+240   T 1076  ☑ 1076 
Q 73+53     T 126   ☑ 126  
Q 434+11    T 445   ☑ 445  
Q 1104+38   T 1142  ☑ 1142 
Q 6320+69   T 6389  ☑ 6389 
Q 7415+0    T 7415  ☑ 7415 
Q 415+397   T 812   ☑ 812  
Q 177+93    T 270   ☑ 270  
Q 7169+6504 T 13673 ☒ 13672
Q 94+48     T 142   ☑ 142  

--------------------------------------------------
Iteration 27
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0353 - acc: 0.9885 - val_loss: 0.0792 - val_acc: 0.9719
Q 994+19    T 1013  ☒ 1003 
Q 6402+4    T 6406  ☑ 6406 
Q 309+80    T 389   ☑ 389  
Q 8156+3    T 8159  ☑ 8159 
Q 2353+459  T 2812  ☑ 2812 
Q 550+79    T 629   ☑ 629  
Q 3319+284  T 3603  ☑ 3603 
Q 5191+19   T 5210  ☒ 5200 
Q 5100+698  T 5798  ☑ 5798 
Q 6611+4    T 6615  ☑ 6615 

--------------------------------------------------
Iteration 28
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - 

40500/40500 [==============================] - 11s 279us/step - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0119 - val_acc: 0.9956
Q 1770+400  T 2170  ☑ 2170 
Q 594+24    T 618   ☑ 618  
Q 8232+922  T 9154  ☑ 9154 
Q 914+2     T 916   ☑ 916  
Q 6289+5142 T 11431 ☑ 11431
Q 5884+96   T 5980  ☑ 5980 
Q 3121+738  T 3859  ☑ 3859 
Q 1770+50   T 1820  ☑ 1820 
Q 6744+208  T 6952  ☑ 6952 
Q 2465+51   T 2516  ☑ 2516 

--------------------------------------------------
Iteration 41
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0092 - acc: 0.9972 - val_loss: 0.0675 - val_acc: 0.9799
Q 2358+1    T 2359  ☒ 2358 
Q 8243+214  T 8457  ☑ 8457 
Q 371+73    T 444   ☑ 444  
Q 152+21    T 173   ☑ 173  
Q 962+138   T 1100  ☑ 1100 
Q 967+184   T 1151  ☒ 1150 
Q 9826+83   T 9909  ☑ 9909 
Q 670+5     T 675   ☑ 675  
Q 161+93    T 254   ☑ 254  
Q 8370+8    T 8378  ☑ 8378 

--------------------------------------------------
Iteratio

40500/40500 [==============================] - 11s 279us/step - loss: 8.7420e-05 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 0.9982
Q 7877+5    T 7882  ☑ 7882 
Q 252+51    T 303   ☑ 303  
Q 8395+84   T 8479  ☑ 8479 
Q 5569+5    T 5574  ☑ 5574 
Q 1892+802  T 2694  ☑ 2694 
Q 7331+61   T 7392  ☑ 7392 
Q 7686+3    T 7689  ☑ 7689 
Q 6556+22   T 6578  ☑ 6578 
Q 944+74    T 1018  ☑ 1018 
Q 7939+9    T 7948  ☑ 7948 

--------------------------------------------------
Iteration 54
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 7.4747e-05 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 0.9982
Q 704+643   T 1347  ☑ 1347 
Q 759+512   T 1271  ☑ 1271 
Q 5595+26   T 5621  ☑ 5621 
Q 594+81    T 675   ☑ 675  
Q 993+23    T 1016  ☑ 1016 
Q 449+5     T 454   ☑ 454  
Q 8252+22   T 8274  ☑ 8274 
Q 513+12    T 525   ☑ 525  
Q 7810+2750 T 10560 ☑ 10560
Q 9617+5    T 9622  ☑ 9622 

--------------------------------------------------


40500/40500 [==============================] - 11s 278us/step - loss: 1.1445e-05 - acc: 1.0000 - val_loss: 0.0042 - val_acc: 0.9984
Q 4204+429  T 4633  ☑ 4633 
Q 524+500   T 1024  ☑ 1024 
Q 7403+75   T 7478  ☑ 7478 
Q 4145+7    T 4152  ☑ 4152 
Q 3549+1    T 3550  ☑ 3550 
Q 8953+2    T 8955  ☑ 8955 
Q 71+57     T 128   ☑ 128  
Q 686+4     T 690   ☑ 690  
Q 940+228   T 1168  ☑ 1168 
Q 947+163   T 1110  ☑ 1110 

--------------------------------------------------
Iteration 67
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 9.7708e-06 - acc: 1.0000 - val_loss: 0.0040 - val_acc: 0.9985
Q 856+79    T 935   ☑ 935  
Q 4375+5    T 4380  ☑ 4380 
Q 8146+7675 T 15821 ☑ 15821
Q 921+45    T 966   ☑ 966  
Q 9043+4527 T 13570 ☑ 13570
Q 726+93    T 819   ☑ 819  
Q 2539+8    T 2547  ☑ 2547 
Q 3772+64   T 3836  ☑ 3836 
Q 7149+561  T 7710  ☑ 7710 
Q 587+584   T 1171  ☑ 1171 

--------------------------------------------------


40500/40500 [==============================] - 11s 278us/step - loss: 1.5766e-06 - acc: 1.0000 - val_loss: 0.0041 - val_acc: 0.9985
Q 1772+68   T 1840  ☑ 1840 
Q 7135+449  T 7584  ☑ 7584 
Q 752+138   T 890   ☑ 890  
Q 2892+532  T 3424  ☑ 3424 
Q 2826+1634 T 4460  ☑ 4460 
Q 2437+24   T 2461  ☑ 2461 
Q 4434+19   T 4453  ☑ 4453 
Q 84+18     T 102   ☑ 102  
Q 6455+4    T 6459  ☑ 6459 
Q 6661+458  T 7119  ☑ 7119 

--------------------------------------------------
Iteration 80
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - loss: 1.3779e-06 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 0.9987
Q 3255+4    T 3259  ☑ 3259 
Q 5233+25   T 5258  ☑ 5258 
Q 6809+125  T 6934  ☑ 6934 
Q 7543+99   T 7642  ☑ 7642 
Q 6138+647  T 6785  ☑ 6785 
Q 466+416   T 882   ☑ 882  
Q 629+77    T 706   ☑ 706  
Q 2063+2    T 2065  ☑ 2065 
Q 9815+8    T 9823  ☑ 9823 
Q 866+541   T 1407  ☑ 1407 

--------------------------------------------------


40500/40500 [==============================] - 11s 279us/step - loss: 3.4732e-07 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 0.9988
Q 842+632   T 1474  ☑ 1474 
Q 1136+265  T 1401  ☑ 1401 
Q 9354+0    T 9354  ☑ 9354 
Q 9964+415  T 10379 ☑ 10379
Q 299+142   T 441   ☑ 441  
Q 6372+82   T 6454  ☑ 6454 
Q 2423+1    T 2424  ☑ 2424 
Q 281+93    T 374   ☑ 374  
Q 5895+4    T 5899  ☑ 5899 
Q 855+5     T 860   ☑ 860  

--------------------------------------------------
Iteration 93
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 3.1932e-07 - acc: 1.0000 - val_loss: 0.0040 - val_acc: 0.9987
Q 2870+16   T 2886  ☑ 2886 
Q 9653+7926 T 17579 ☑ 17579
Q 4690+271  T 4961  ☑ 4961 
Q 7807+735  T 8542  ☑ 8542 
Q 2639+49   T 2688  ☑ 2688 
Q 871+11    T 882   ☑ 882  
Q 2322+682  T 3004  ☑ 3004 
Q 6756+90   T 6846  ☑ 6846 
Q 9009+838  T 9847  ☑ 9847 
Q 187+183   T 370   ☑ 370  

--------------------------------------------------


### Analysis
* Only addition
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 15 epoch

In [13]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS
chars = '0123456789- '
DATA_SIZE = 30000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 30000
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 6s 291us/step - loss: 2.2820 - acc: 0.1586 - val_loss: 2.1163 - val_acc: 0.2185
Q 985-361 T 624 ☒ 20 
Q 532-28  T 504 ☒ 434
Q 553-341 T 212 ☒ 40 
Q 849-1   T 848 ☒ 884
Q 303-6   T 297 ☒ 334
Q 617-491 T 126 ☒ 20 
Q 414-3   T 411 ☒ 434
Q 152-27  T 125 ☒ 10 
Q 298-39  T 259 ☒ 824
Q 623-44  T 579 ☒ 634

--------------------------------------------------
Iteration 1
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/step - loss: 1.8779 - acc: 0.2990 - val_loss: 1.8300 - val_acc: 0.3244
Q 647-64  T 583 ☒ 574
Q 968-54  T 914 ☒ 944
Q 372-308 T 64  ☒ 20 
Q 731-59  T 672 ☒ 691
Q 767-506 T 261 ☒ 16 
Q 556-79  T 477 ☒ 574
Q 450-25  T 425 ☒ 474
Q 992-531 T 461 ☒ 16 
Q 75-67   T 8   ☒ 1  

20250/20250 [==============================] - 4s 184us/step - loss: 0.1428 - acc: 0.9575 - val_loss: 0.1610 - val_acc: 0.9476
Q 400-57  T 343 ☑ 343
Q 845-54  T 791 ☑ 791
Q 670-121 T 549 ☑ 549
Q 68-25   T 43  ☑ 43 
Q 698-670 T 28  ☒ 17 
Q 775-301 T 474 ☑ 474
Q 655-460 T 195 ☑ 195
Q 764-52  T 712 ☑ 712
Q 94-47   T 47  ☑ 47 
Q 539-97  T 442 ☑ 442

--------------------------------------------------
Iteration 16
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/step - loss: 0.1368 - acc: 0.9578 - val_loss: 0.1741 - val_acc: 0.9425
Q 451-3   T 448 ☑ 448
Q 441-29  T 412 ☑ 412
Q 170-22  T 148 ☑ 148
Q 352-349 T 3   ☒ 8  
Q 779-266 T 513 ☑ 513
Q 618-406 T 212 ☑ 212
Q 375-159 T 216 ☑ 216
Q 818-31  T 787 ☑ 787
Q 384-325 T 59  ☒ 58 
Q 895-53  T 842 ☑ 842

--------------------------------------------------
Iteration 17
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.0313 - acc: 0.9951 - val_loss: 0.0408 - val_acc: 0.9899
Q 362-41  T 321 ☑ 321
Q 114-53  T 61  ☑ 61 
Q 323-20  T 303 ☑ 303
Q 196-128 T 68  ☑ 68 
Q 566-35  T 531 ☑ 531
Q 361-39  T 322 ☑ 322
Q 111-5   T 106 ☑ 106
Q 682-15  T 667 ☑ 667
Q 550-13  T 537 ☑ 537
Q 799-2   T 797 ☑ 797

--------------------------------------------------
Iteration 31
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/step - loss: 0.0281 - acc: 0.9955 - val_loss: 0.0445 - val_acc: 0.9876
Q 971-50  T 921 ☑ 921
Q 353-29  T 324 ☑ 324
Q 696-54  T 642 ☑ 642
Q 933-284 T 649 ☑ 649
Q 368-6   T 362 ☑ 362
Q 944-30  T 914 ☑ 914
Q 856-87  T 769 ☑ 769
Q 334-104 T 230 ☑ 230
Q 770-23  T 747 ☑ 747
Q 447-268 T 179 ☑ 179

--------------------------------------------------
Iteration 32
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.1417 - acc: 0.9551 - val_loss: 0.0616 - val_acc: 0.9807
Q 544-8   T 536 ☑ 536
Q 902-24  T 878 ☑ 878
Q 459-69  T 390 ☑ 390
Q 642-325 T 317 ☑ 317
Q 411-48  T 363 ☑ 363
Q 880-60  T 820 ☑ 820
Q 978-396 T 582 ☑ 582
Q 717-51  T 666 ☑ 666
Q 478-356 T 122 ☑ 122
Q 814-477 T 337 ☑ 337

--------------------------------------------------
Iteration 46
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 185us/step - loss: 0.0275 - acc: 0.9938 - val_loss: 0.0462 - val_acc: 0.9846
Q 287-54  T 233 ☑ 233
Q 917-20  T 897 ☑ 897
Q 835-401 T 434 ☑ 434
Q 146-92  T 54  ☒ 64 
Q 561-3   T 558 ☑ 558
Q 975-726 T 249 ☑ 249
Q 71-69   T 2   ☑ 2  
Q 699-47  T 652 ☑ 652
Q 578-56  T 522 ☑ 522
Q 48-37   T 11  ☑ 11 

--------------------------------------------------
Iteration 47
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/st

20250/20250 [==============================] - 4s 183us/step - loss: 0.0045 - acc: 0.9997 - val_loss: 0.0198 - val_acc: 0.9948
Q 330-4   T 326 ☑ 326
Q 917-20  T 897 ☑ 897
Q 235-37  T 198 ☑ 198
Q 675-75  T 600 ☑ 600
Q 152-27  T 125 ☑ 125
Q 448-0   T 448 ☑ 448
Q 741-14  T 727 ☑ 727
Q 973-848 T 125 ☑ 125
Q 917-239 T 678 ☑ 678
Q 418-19  T 399 ☑ 399

--------------------------------------------------
Iteration 61
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/step - loss: 0.0044 - acc: 0.9996 - val_loss: 0.0224 - val_acc: 0.9938
Q 640-477 T 163 ☑ 163
Q 864-38  T 826 ☑ 826
Q 762-58  T 704 ☑ 704
Q 733-184 T 549 ☑ 549
Q 306-64  T 242 ☑ 242
Q 731-4   T 727 ☑ 727
Q 977-866 T 111 ☑ 111
Q 582-423 T 159 ☑ 159
Q 123-82  T 41  ☑ 41 
Q 705-21  T 684 ☑ 684

--------------------------------------------------
Iteration 62
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/st

20250/20250 [==============================] - 4s 183us/step - loss: 0.0501 - acc: 0.9845 - val_loss: 0.0296 - val_acc: 0.9917
Q 268-0   T 268 ☑ 268
Q 690-455 T 235 ☑ 235
Q 383-223 T 160 ☑ 160
Q 957-704 T 253 ☑ 253
Q 741-9   T 732 ☑ 732
Q 425-47  T 378 ☑ 378
Q 615-89  T 526 ☑ 526
Q 547-69  T 478 ☑ 478
Q 817-636 T 181 ☑ 181
Q 129-4   T 125 ☑ 125

--------------------------------------------------
Iteration 76
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/step - loss: 0.0082 - acc: 0.9990 - val_loss: 0.0297 - val_acc: 0.9905
Q 395-6   T 389 ☑ 389
Q 547-336 T 211 ☑ 211
Q 360-39  T 321 ☑ 321
Q 561-3   T 558 ☑ 558
Q 408-26  T 382 ☑ 382
Q 705-7   T 698 ☑ 698
Q 379-68  T 311 ☑ 311
Q 87-49   T 38  ☑ 38 
Q 835-42  T 793 ☑ 793
Q 731-4   T 727 ☑ 727

--------------------------------------------------
Iteration 77
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0173 - val_acc: 0.9961
Q 534-102 T 432 ☑ 432
Q 651-86  T 565 ☑ 565
Q 562-1   T 561 ☑ 561
Q 960-6   T 954 ☑ 954
Q 533-69  T 464 ☑ 464
Q 230-2   T 228 ☑ 228
Q 816-435 T 381 ☑ 381
Q 577-5   T 572 ☑ 572
Q 835-401 T 434 ☑ 434
Q 284-2   T 282 ☑ 282

--------------------------------------------------
Iteration 91
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0172 - val_acc: 0.9957
Q 381-57  T 324 ☑ 324
Q 500-61  T 439 ☑ 439
Q 696-38  T 658 ☑ 658
Q 476-88  T 388 ☑ 388
Q 216-58  T 158 ☑ 158
Q 85-13   T 72  ☑ 72 
Q 753-616 T 137 ☑ 137
Q 946-330 T 616 ☑ 616
Q 463-26  T 437 ☑ 437
Q 127-49  T 78  ☑ 78 

--------------------------------------------------
Iteration 92
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/st

### Analysis
* Only subtraction
* 3 digits
* Training size: 20.25K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 32 epoch

In [14]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
ansmaxlen = DIGITS
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 12s 302us/step - loss: 1.9678 - acc: 0.2708 - val_loss: 1.5769 - val_acc: 0.3854
Q 4403-473  T 3930 ☒ 4335
Q 5634-3    T 5631 ☒ 5533
Q 3566-16   T 3550 ☒ 3555
Q 7056-484  T 6572 ☒ 7333
Q 379-84    T 295  ☒ 377 
Q 8727-6386 T 2341 ☒ 233 
Q 7824-795  T 7029 ☒ 7033
Q 2047-806  T 1241 ☒ 233 
Q 3186-154  T 3032 ☒ 3005
Q 935-83    T 852  ☒ 833 

--------------------------------------------------
Iteration 1
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 235us/step - loss: 1.3457 - acc: 0.4830 - val_loss: 1.2326 - val_acc: 0.5261
Q 4537-2    T 4535 ☒ 4534
Q 7422-79   T 7343 ☒ 7365
Q 418-16    T 402  ☒ 408 
Q 5845-28   T 5817 ☒ 5825
Q 4246-590  T 3656 ☒ 3855
Q 8429-4641 T 3788 ☒ 4525

40500/40500 [==============================] - 9s 234us/step - loss: 0.0776 - acc: 0.9744 - val_loss: 0.1304 - val_acc: 0.9524
Q 423-2     T 421  ☑ 421 
Q 1206-207  T 999  ☒ 109 
Q 7656-4896 T 2760 ☑ 2760
Q 2805-303  T 2502 ☑ 2502
Q 7921-6227 T 1694 ☒ 1794
Q 9356-6925 T 2431 ☑ 2431
Q 5568-2    T 5566 ☑ 5566
Q 883-771   T 112  ☑ 112 
Q 5051-4316 T 735  ☑ 735 
Q 531-19    T 512  ☑ 512 

--------------------------------------------------
Iteration 15
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 234us/step - loss: 0.0618 - acc: 0.9802 - val_loss: 0.1194 - val_acc: 0.9643
Q 5853-1    T 5852 ☑ 5852
Q 1781-846  T 935  ☑ 935 
Q 4973-881  T 4092 ☒ 4192
Q 1691-75   T 1616 ☑ 1616
Q 335-64    T 271  ☑ 271 
Q 4119-43   T 4076 ☑ 4076
Q 4981-4    T 4977 ☑ 4977
Q 868-265   T 603  ☑ 603 
Q 568-275   T 293  ☒ 393 
Q 9141-98   T 9043 ☑ 9043

--------------------------------------------------
Iteration 16
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 232us/step - loss: 0.0460 - acc: 0.9853 - val_loss: 0.0835 - val_acc: 0.9719
Q 964-906   T 58   ☑ 58  
Q 281-88    T 193  ☑ 193 
Q 4803-3    T 4800 ☑ 4800
Q 9831-341  T 9490 ☒ 9390
Q 893-84    T 809  ☑ 809 
Q 416-28    T 388  ☑ 388 
Q 2640-493  T 2147 ☑ 2147
Q 5618-8    T 5610 ☑ 5610
Q 751-196   T 555  ☑ 555 
Q 734-4     T 730  ☑ 730 

--------------------------------------------------
Iteration 29
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 235us/step - loss: 0.0167 - acc: 0.9961 - val_loss: 0.0321 - val_acc: 0.9903
Q 920-63    T 857  ☑ 857 
Q 8766-2346 T 6420 ☑ 6420
Q 4080-6    T 4074 ☑ 4074
Q 4427-74   T 4353 ☑ 4353
Q 3967-4    T 3963 ☑ 3963
Q 1691-75   T 1616 ☑ 1616
Q 8226-858  T 7368 ☑ 7368
Q 6778-440  T 6338 ☑ 6338
Q 6735-342  T 6393 ☑ 6393
Q 4257-2465 T 1792 ☑ 1792

--------------------------------------------------
Iteration 30
Train on 40500 samples, validate on 

40500/40500 [==============================] - 9s 233us/step - loss: 0.0709 - acc: 0.9787 - val_loss: 0.0525 - val_acc: 0.9846
Q 9713-748  T 8965 ☑ 8965
Q 5978-927  T 5051 ☑ 5051
Q 318-280   T 38   ☑ 38  
Q 8381-1163 T 7218 ☑ 7218
Q 8862-3505 T 5357 ☑ 5357
Q 750-67    T 683  ☑ 683 
Q 9285-9    T 9276 ☑ 9276
Q 5326-74   T 5252 ☑ 5252
Q 301-89    T 212  ☑ 212 
Q 5845-28   T 5817 ☑ 5817

--------------------------------------------------
Iteration 43
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 234us/step - loss: 0.0190 - acc: 0.9946 - val_loss: 0.0417 - val_acc: 0.9868
Q 7926-12   T 7914 ☑ 7914
Q 2406-421  T 1985 ☑ 1985
Q 991-296   T 695  ☑ 695 
Q 4789-39   T 4750 ☑ 4750
Q 2790-85   T 2705 ☑ 2705
Q 1076-4    T 1072 ☑ 1072
Q 225-183   T 42   ☑ 42  
Q 481-98    T 383  ☑ 383 
Q 746-7     T 739  ☑ 739 
Q 2141-30   T 2111 ☑ 2111

--------------------------------------------------
Iteration 44
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 234us/step - loss: 7.0599e-04 - acc: 1.0000 - val_loss: 0.0251 - val_acc: 0.9943
Q 883-48    T 835  ☑ 835 
Q 7946-450  T 7496 ☑ 7496
Q 6570-22   T 6548 ☑ 6548
Q 964-906   T 58   ☑ 58  
Q 2456-990  T 1466 ☑ 1466
Q 4443-5    T 4438 ☑ 4438
Q 8284-95   T 8189 ☑ 8189
Q 282-0     T 282  ☑ 282 
Q 705-438   T 267  ☑ 267 
Q 2999-36   T 2963 ☑ 2963

--------------------------------------------------
Iteration 57
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 234us/step - loss: 0.0144 - acc: 0.9968 - val_loss: 0.6024 - val_acc: 0.8615
Q 6442-15   T 6427 ☑ 6427
Q 1462-51   T 1411 ☑ 1411
Q 2623-8    T 2615 ☒ 2715
Q 76-72     T 4    ☒ 3   
Q 501-42    T 459  ☑ 459 
Q 5209-11   T 5198 ☒ 5298
Q 7946-450  T 7496 ☒ 7495
Q 156-9     T 147  ☑ 147 
Q 193-57    T 136  ☑ 136 
Q 4853-88   T 4765 ☑ 4765

--------------------------------------------------
Iteration 58
Train on 40500 samples, validate 

40500/40500 [==============================] - 9s 234us/step - loss: 0.0019 - acc: 0.9997 - val_loss: 0.0427 - val_acc: 0.9888
Q 5584-49   T 5535 ☑ 5535
Q 176-0     T 176  ☑ 176 
Q 4115-2    T 4113 ☑ 4113
Q 3316-14   T 3302 ☑ 3302
Q 4426-397  T 4029 ☑ 4029
Q 9111-3    T 9108 ☑ 9108
Q 2542-76   T 2466 ☑ 2466
Q 2185-2    T 2183 ☑ 2183
Q 4027-636  T 3391 ☑ 3391
Q 6653-5    T 6648 ☑ 6648

--------------------------------------------------
Iteration 71
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 233us/step - loss: 0.0307 - acc: 0.9908 - val_loss: 0.0377 - val_acc: 0.9898
Q 774-138   T 636  ☑ 636 
Q 428-60    T 368  ☑ 368 
Q 8714-16   T 8698 ☑ 8698
Q 5626-1    T 5625 ☑ 5625
Q 970-524   T 446  ☑ 446 
Q 6580-6575 T 5    ☒ 7   
Q 4776-78   T 4698 ☑ 4698
Q 1059-35   T 1024 ☑ 1024
Q 3505-0    T 3505 ☑ 3505
Q 1744-758  T 986  ☑ 986 

--------------------------------------------------
Iteration 72
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 10s 235us/step - loss: 1.8643e-04 - acc: 1.0000 - val_loss: 0.0217 - val_acc: 0.9951
Q 218-42    T 176  ☑ 176 
Q 7884-6618 T 1266 ☑ 1266
Q 687-68    T 619  ☑ 619 
Q 583-9     T 574  ☑ 574 
Q 9073-9    T 9064 ☑ 9064
Q 5296-688  T 4608 ☑ 4608
Q 7091-2542 T 4549 ☑ 4549
Q 7129-56   T 7073 ☑ 7073
Q 5418-5    T 5413 ☑ 5413
Q 7234-5    T 7229 ☑ 7229

--------------------------------------------------
Iteration 85
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 233us/step - loss: 1.5922e-04 - acc: 1.0000 - val_loss: 0.0218 - val_acc: 0.9951
Q 63-19     T 44   ☑ 44  
Q 953-627   T 326  ☑ 326 
Q 2609-631  T 1978 ☑ 1978
Q 247-4     T 243  ☑ 243 
Q 9862-5    T 9857 ☑ 9857
Q 2185-2    T 2183 ☑ 2183
Q 1964-446  T 1518 ☑ 1518
Q 1318-459  T 859  ☑ 859 
Q 5384-4    T 5380 ☑ 5380
Q 800-3     T 797  ☑ 797 

--------------------------------------------------
Iteration 86
Train on 40500 samples, vali

40500/40500 [==============================] - 9s 234us/step - loss: 1.1414e-04 - acc: 1.0000 - val_loss: 0.0224 - val_acc: 0.9949
Q 8719-5    T 8714 ☑ 8714
Q 198-89    T 109  ☑ 109 
Q 6749-50   T 6699 ☑ 6699
Q 7175-3114 T 4061 ☑ 4061
Q 5724-879  T 4845 ☑ 4845
Q 617-97    T 520  ☑ 520 
Q 572-44    T 528  ☑ 528 
Q 192-41    T 151  ☑ 151 
Q 3988-210  T 3778 ☑ 3778
Q 8015-35   T 7980 ☑ 7980

--------------------------------------------------
Iteration 99
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 234us/step - loss: 9.9716e-05 - acc: 1.0000 - val_loss: 0.0223 - val_acc: 0.9949
Q 6342-186  T 6156 ☑ 6156
Q 2404-83   T 2321 ☑ 2321
Q 8122-1024 T 7098 ☑ 7098
Q 8606-7673 T 933  ☑ 933 
Q 505-8     T 497  ☑ 497 
Q 3329-4    T 3325 ☑ 3325
Q 2299-68   T 2231 ☑ 2231
Q 282-0     T 282  ☑ 282 
Q 862-121   T 741  ☑ 741 
Q 5732-220  T 5512 ☑ 5512


### Analysis
* Only subtraction
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 29 epoch

In [15]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS
chars = '0123456789+- '
ansmaxlen = DIGITS + 1
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen, combime=True)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
model.fit(x_train, y_train, batch_size=BATCH_SIZE)
#     model.build()
model.summary()
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...
Epoch 1/1
40500/40500 [==============================] - 13s 315us/step - loss: 1.7910 - acc: 0.3667
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 512)               1077248   
_________________________________________________________________
reshape_5 (Reshape)          (None, 4, 128)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 4, 512)            1312768   
_________________________________________________________________
dense_5 (Dense)              (None, 4, 13)             6669      
Total params: 2,396,685
Trainable params: 2,396,685
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 40500 s

40500/40500 [==============================] - 10s 258us/step - loss: 0.6411 - acc: 0.7541 - val_loss: 0.6850 - val_acc: 0.7368
Q 512-32+9    T 489  ☒ 492 
Q 73+46-5     T 114  ☑ 114 
Q 648+119+28  T 795  ☑ 795 
Q 283-77-3    T 203  ☑ 203 
Q 331+74-5    T 400  ☑ 400 
Q 390+65+1    T 456  ☒ 452 
Q 86-81+1     T 6    ☒ 1   
Q 71+15+0     T 86   ☑ 86  
Q 366-62-7    T 297  ☑ 297 
Q 672-250-7   T 415  ☒ 414 

--------------------------------------------------
Iteration 13
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - loss: 0.5894 - acc: 0.7745 - val_loss: 0.6398 - val_acc: 0.7527
Q 914+76+8    T 998  ☒ 995 
Q 383-80+0    T 303  ☑ 303 
Q 549+76-43   T 582  ☒ 580 
Q 258-55+9    T 212  ☒ 210 
Q 601-8+0     T 593  ☒ 592 
Q 440-21+8    T 427  ☒ 428 
Q 745+212-24  T 933  ☑ 933 
Q 438-248-53  T 137  ☒ 136 
Q 46+45+0     T 91   ☒ 90  
Q 863+634-24  T 1473 ☒ 1471

--------------------------------------------------
Iteratio

Q 899+468-95  T 1272 ☒ 1383
Q 766+163-135 T 794  ☒ 795 
Q 291+76+1    T 368  ☑ 368 
Q 646-6+6     T 646  ☑ 646 
Q 93+54+1     T 148  ☑ 148 
Q 943+7+7     T 957  ☑ 957 
Q 942-548+28  T 422  ☒ 421 
Q 128+93+3    T 224  ☑ 224 
Q 978-625+44  T 397  ☒ 395 
Q 805-519+22  T 308  ☒ 309 

--------------------------------------------------
Iteration 26
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - loss: 0.1786 - acc: 0.9365 - val_loss: 0.2516 - val_acc: 0.9080
Q 532-4+0     T 528  ☑ 528 
Q 824+75-5    T 894  ☑ 894 
Q 793-41+30   T 782  ☑ 782 
Q 660+127-40  T 747  ☑ 747 
Q 85-46+9     T 48   ☑ 48  
Q 617-13+12   T 616  ☑ 616 
Q 450-52+15   T 413  ☑ 413 
Q 448+271-7   T 712  ☑ 712 
Q 962+643+356 T 1961 ☑ 1961
Q 918+372+11  T 1301 ☑ 1301

--------------------------------------------------
Iteration 27
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - 

40500/40500 [==============================] - 10s 259us/step - loss: 0.0895 - acc: 0.9689 - val_loss: 0.1607 - val_acc: 0.9459
Q 225-88+1    T 138  ☑ 138 
Q 423-48+2    T 377  ☑ 377 
Q 868-33+18   T 853  ☑ 853 
Q 654-6+2     T 650  ☑ 650 
Q 96+13-4     T 105  ☑ 105 
Q 405-33+6    T 378  ☑ 378 
Q 830+83-3    T 910  ☑ 910 
Q 88+6-5      T 89   ☑ 89  
Q 42-41+6     T 7    ☑ 7   
Q 126+51-5    T 172  ☑ 172 

--------------------------------------------------
Iteration 40
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - loss: 0.0579 - acc: 0.9814 - val_loss: 0.1941 - val_acc: 0.9347
Q 98-36-6     T 56   ☑ 56  
Q 836+498-203 T 1131 ☒ 1031
Q 523+67-4    T 586  ☑ 586 
Q 54-22+0     T 32   ☑ 32  
Q 98-80+9     T 27   ☑ 27  
Q 128+23-5    T 146  ☑ 146 
Q 38+32+0     T 70   ☑ 70  
Q 901-179+2   T 724  ☑ 724 
Q 657+332+4   T 993  ☑ 993 
Q 601+12-9    T 604  ☒ 603 

--------------------------------------------------
Iteratio

Q 973-905+8   T 76   ☒ 87  
Q 391+6-4     T 393  ☑ 393 
Q 385+86+1    T 472  ☑ 472 
Q 76+14-6     T 84   ☑ 84  
Q 240+7-2     T 245  ☑ 245 
Q 98+83+33    T 214  ☑ 214 
Q 86-32-11    T 43   ☑ 43  
Q 108-3-2     T 103  ☑ 103 
Q 858+44-30   T 872  ☑ 872 
Q 869-417-18  T 434  ☑ 434 

--------------------------------------------------
Iteration 53
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - loss: 0.0392 - acc: 0.9875 - val_loss: 0.1882 - val_acc: 0.9409
Q 165+64+5    T 234  ☑ 234 
Q 883+386-5   T 1264 ☑ 1264
Q 903+242-5   T 1140 ☑ 1140
Q 857+418-79  T 1196 ☒ 1106
Q 481+162+91  T 734  ☒ 736 
Q 942-61+8    T 889  ☒ 899 
Q 27-25+5     T 7    ☑ 7   
Q 951+92+7    T 1050 ☑ 1050
Q 96-60-23    T 13   ☑ 13  
Q 615-45-0    T 570  ☑ 570 

--------------------------------------------------
Iteration 54
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 258us/step - 

40500/40500 [==============================] - 11s 260us/step - loss: 0.0315 - acc: 0.9904 - val_loss: 0.2857 - val_acc: 0.9171
Q 129+8+6     T 143  ☑ 143 
Q 650+8+6     T 664  ☑ 664 
Q 715-9-9     T 697  ☑ 697 
Q 48-18-4     T 26   ☑ 26  
Q 510+53+5    T 568  ☑ 568 
Q 109+51+1    T 161  ☑ 161 
Q 648+265+1   T 914  ☑ 914 
Q 62-47+8     T 23   ☑ 23  
Q 985-38+5    T 952  ☑ 952 
Q 759+495+2   T 1256 ☑ 1256

--------------------------------------------------
Iteration 67
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 258us/step - loss: 0.0487 - acc: 0.9839 - val_loss: 0.1576 - val_acc: 0.9531
Q 745-647-21  T 77   ☒ 78  
Q 739+69+2    T 810  ☑ 810 
Q 811+81+40   T 932  ☑ 932 
Q 686-3+1     T 684  ☑ 684 
Q 291-7+7     T 291  ☑ 291 
Q 722-178+5   T 549  ☒ 559 
Q 692-296+45  T 441  ☑ 441 
Q 883+386-5   T 1264 ☑ 1264
Q 578-90+46   T 534  ☑ 534 
Q 876+347+5   T 1228 ☑ 1228

--------------------------------------------------
Iteratio

Q 623+8-3     T 628  ☑ 628 
Q 95-83+7     T 19   ☑ 19  
Q 233-23-8    T 202  ☑ 202 
Q 72+63+46    T 181  ☒ 191 
Q 672-250-7   T 415  ☑ 415 
Q 318+159+30  T 507  ☑ 507 
Q 830-572-39  T 219  ☒ 210 
Q 51-40+9     T 20   ☑ 20  
Q 421+4-4     T 421  ☑ 421 
Q 891-30+5    T 866  ☑ 866 

--------------------------------------------------
Iteration 80
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 259us/step - loss: 0.0162 - acc: 0.9957 - val_loss: 0.1516 - val_acc: 0.9576
Q 35-13-4     T 18   ☑ 18  
Q 807-98-11   T 698  ☑ 698 
Q 971+5+1     T 977  ☑ 977 
Q 823-695+51  T 179  ☒ 189 
Q 834+30-4    T 860  ☑ 860 
Q 964+960+336 T 2260 ☒ 2270
Q 778+166+41  T 985  ☑ 985 
Q 571-390+9   T 190  ☑ 190 
Q 91-65+9     T 35   ☑ 35  
Q 894-95-8    T 791  ☑ 791 

--------------------------------------------------
Iteration 81
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 258us/step - 

40500/40500 [==============================] - 10s 258us/step - loss: 0.0227 - acc: 0.9931 - val_loss: 0.1836 - val_acc: 0.9487
Q 920-2+2     T 920  ☑ 920 
Q 623-60-16   T 547  ☑ 547 
Q 135-14-1    T 120  ☑ 120 
Q 708-552-7   T 149  ☑ 149 
Q 391+4-1     T 394  ☑ 394 
Q 584+98-9    T 673  ☑ 673 
Q 882+739+12  T 1633 ☒ 1634
Q 229-78+60   T 211  ☑ 211 
Q 867-222-32  T 613  ☑ 613 
Q 484+36-20   T 500  ☒ 400 

--------------------------------------------------
Iteration 94
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 10s 258us/step - loss: 0.0246 - acc: 0.9925 - val_loss: 0.2009 - val_acc: 0.9455
Q 93-58+8     T 43   ☑ 43  
Q 99-6-5      T 88   ☑ 88  
Q 758-9+0     T 749  ☑ 749 
Q 886-668-1   T 217  ☑ 217 
Q 85-75-1     T 9    ☑ 9   
Q 603-78+2    T 527  ☑ 527 
Q 687+27-0    T 714  ☑ 714 
Q 925+80+7    T 1012 ☑ 1012
Q 323-69-6    T 248  ☑ 248 
Q 240+229-6   T 463  ☑ 463 

--------------------------------------------------
Iteratio

### Analysis
* addition & subtraction
* 3 digits
* Training size: 30.375K
* Total training epoch: 100 epoch
* Best accuracy: 96.52%

In [16]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+- '
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen, combime=True)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 18s 432us/step - loss: 1.7848 - acc: 0.3646 - val_loss: 1.6987 - val_acc: 0.3730
Q 9283-34-8      T 9241  ☒ 1440 
Q 2787-202-6     T 2579  ☒ 1440 
Q 3175+718-0     T 3893  ☒ 1440 
Q 449+271-2      T 718   ☒ 144  
Q 5490+970-221   T 6239  ☒ 1040 
Q 495+93+19      T 607   ☒ 144  
Q 4028-939-916   T 2173  ☒ 1440 
Q 68+9-9         T 68    ☒ 14   
Q 3885-88+6      T 3803  ☒ 1440 
Q 9544-527+276   T 9293  ☒ 1040 

--------------------------------------------------
Iteration 1
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 1.5257 - acc: 0.4328 - val_loss: 1.4213 - val_acc: 0.4642
Q 8542-7119-856  T 567   ☒ 4369 
Q 863-139+7      T 731   ☒ 606  
Q 7592+727+61    T 

40500/40500 [==============================] - 14s 344us/step - loss: 0.5302 - acc: 0.7983 - val_loss: 0.5527 - val_acc: 0.7850
Q 9969+1501-10   T 11460 ☒ 12440
Q 320+8+1        T 329   ☒ 339  
Q 372+6-0        T 378   ☑ 378  
Q 594-78+2       T 518   ☒ 528  
Q 6873-88-6      T 6779  ☒ 6789 
Q 9847-398-27    T 9422  ☑ 9422 
Q 2897-27+1      T 2871  ☑ 2871 
Q 88-47+1        T 42    ☑ 42   
Q 73+37-0        T 110   ☑ 110  
Q 323-93-71      T 159   ☒ 16   

--------------------------------------------------
Iteration 13
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.4735 - acc: 0.8197 - val_loss: 0.5212 - val_acc: 0.7943
Q 9657+544-4     T 10197 ☒ 10188
Q 719+55-3       T 771   ☑ 771  
Q 9264-6-3       T 9255  ☑ 9255 
Q 7327-1497+129  T 5959  ☒ 5972 
Q 6045-2188+4    T 3861  ☒ 3862 
Q 1564-53+0      T 1511  ☑ 1511 
Q 6919-391-1     T 6527  ☒ 6526 
Q 528-286+28     T 270   ☒ 269  
Q 373-54+34      T 353   ☒

40500/40500 [==============================] - 14s 344us/step - loss: 0.1233 - acc: 0.9583 - val_loss: 0.3077 - val_acc: 0.8952
Q 4752-913-66    T 3773  ☑ 3773 
Q 5119-24-8      T 5087  ☑ 5087 
Q 9450+902-553   T 9799  ☒ 9889 
Q 501-74+8       T 435   ☑ 435  
Q 9369+121+1     T 9491  ☒ 9591 
Q 3773+736+5     T 4514  ☒ 4515 
Q 766-71+39      T 734   ☑ 734  
Q 301-61-3       T 237   ☑ 237  
Q 577+30+1       T 608   ☑ 608  
Q 8281-4694-496  T 3091  ☒ 3000 

--------------------------------------------------
Iteration 26
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.0970 - acc: 0.9687 - val_loss: 0.3138 - val_acc: 0.8965
Q 9892+313-67    T 10138 ☒ 10148
Q 5906+281+217   T 6404  ☒ 6408 
Q 4897+3983+2    T 8882  ☑ 8882 
Q 9612+872-16    T 10468 ☒ 10479
Q 568+158-2      T 724   ☑ 724  
Q 347+9-1        T 355   ☑ 355  
Q 590-500-1      T 89    ☑ 89   
Q 9145-110-5     T 9030  ☑ 9030 
Q 569-470-6      T 93    ☑

40500/40500 [==============================] - 14s 344us/step - loss: 0.0605 - acc: 0.9804 - val_loss: 0.4347 - val_acc: 0.8816
Q 195-44-8       T 143   ☑ 143  
Q 2977+2045-10   T 5012  ☒ 5900 
Q 8601-299+72    T 8374  ☒ 8384 
Q 7191-460+13    T 6744  ☑ 6744 
Q 8730+3903+0    T 12633 ☒ 12630
Q 7326+508+9     T 7843  ☑ 7843 
Q 9329+68+8      T 9405  ☒ 9494 
Q 567+64+55      T 686   ☑ 686  
Q 14+9+4         T 27    ☑ 27   
Q 77+67-2        T 142   ☑ 142  

--------------------------------------------------
Iteration 39
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.0473 - acc: 0.9852 - val_loss: 0.3910 - val_acc: 0.8949
Q 6290+94+8      T 6392  ☑ 6392 
Q 9657+544-4     T 10197 ☑ 10197
Q 711+483+73     T 1267  ☒ 1280 
Q 801+21-0       T 822   ☑ 822  
Q 240-70-2       T 168   ☑ 168  
Q 8485-7523+85   T 1047  ☒ 1056 
Q 3672-124+98    T 3646  ☒ 3656 
Q 921-21+0       T 900   ☑ 900  
Q 182-79-4       T 99    ☒

40500/40500 [==============================] - 14s 344us/step - loss: 0.0378 - acc: 0.9882 - val_loss: 0.4915 - val_acc: 0.8823
Q 9430+1843-7    T 11266 ☒ 11265
Q 99+34+31       T 164   ☑ 164  
Q 997+7-3        T 1001  ☑ 1001 
Q 5131+1176-140  T 6167  ☒ 6165 
Q 384-91+81      T 374   ☑ 374  
Q 5866-86-64     T 5716  ☒ 5726 
Q 9236-5138-1    T 4097  ☒ 4198 
Q 608+183-9      T 782   ☑ 782  
Q 4804-273-12    T 4519  ☒ 4529 
Q 142+9-1        T 150   ☑ 150  

--------------------------------------------------
Iteration 52
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.0331 - acc: 0.9898 - val_loss: 0.4415 - val_acc: 0.8937
Q 749-376+9      T 382   ☒ 381  
Q 193+4-2        T 195   ☑ 195  
Q 528-95+8       T 441   ☑ 441  
Q 4845-1487+219  T 3577  ☒ 3575 
Q 8103+713+90    T 8906  ☒ 8916 
Q 5277-3504-424  T 1349  ☒ 1359 
Q 8378+95-8      T 8465  ☑ 8465 
Q 7081-814-57    T 6210  ☒ 6200 
Q 742-72+4       T 674   ☑

40500/40500 [==============================] - 14s 344us/step - loss: 0.0195 - acc: 0.9943 - val_loss: 0.5520 - val_acc: 0.8772
Q 729+9+8        T 746   ☑ 746  
Q 7377-5920-2    T 1455  ☑ 1455 
Q 424-45-15      T 364   ☒ 363  
Q 289+64+1       T 354   ☑ 354  
Q 913+365-58     T 1220  ☑ 1220 
Q 66+30+5        T 101   ☑ 101  
Q 893-754+386    T 525   ☑ 525  
Q 376+62+5       T 443   ☑ 443  
Q 651+96-8       T 739   ☑ 739  
Q 1073-125-0     T 948   ☒ 989  

--------------------------------------------------
Iteration 65
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.0517 - acc: 0.9830 - val_loss: 0.5181 - val_acc: 0.8816
Q 3520+816-0     T 4336  ☑ 4336 
Q 75-39-5        T 31    ☑ 31   
Q 59+5-0         T 64    ☑ 64   
Q 5186-31-0      T 5155  ☑ 5155 
Q 4613+778+340   T 5731  ☒ 5732 
Q 667-92-21      T 554   ☑ 554  
Q 9767+3541+86   T 13394 ☒ 13384
Q 4127+1636+403  T 6166  ☒ 606  
Q 4649+257-2     T 4904  ☑

40500/40500 [==============================] - 14s 344us/step - loss: 0.0143 - acc: 0.9960 - val_loss: 0.5178 - val_acc: 0.8874
Q 8774-5529+10   T 3255  ☑ 3255 
Q 7980-5475+189  T 2694  ☒ 2592 
Q 57-7+3         T 53    ☑ 53   
Q 4341-3-1       T 4337  ☑ 4337 
Q 274+34-13      T 295   ☑ 295  
Q 8028+500-6     T 8522  ☑ 8522 
Q 8447+347+85    T 8879  ☒ 8889 
Q 7705-5+2       T 7702  ☑ 7702 
Q 670+28-1       T 697   ☑ 697  
Q 749-376+9      T 382   ☒ 381  

--------------------------------------------------
Iteration 78
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 345us/step - loss: 0.0109 - acc: 0.9971 - val_loss: 0.5165 - val_acc: 0.8885
Q 8874-6367-25   T 2482  ☑ 2482 
Q 9670+2197-6    T 11861 ☒ 11872
Q 5636-430-3     T 5203  ☑ 5203 
Q 9214-26-9      T 9179  ☑ 9179 
Q 805+773+5      T 1583  ☒ 1582 
Q 7133+213-96    T 7250  ☑ 7250 
Q 9481-767-98    T 8616  ☒ 8608 
Q 960+702-8      T 1654  ☒ 1643 
Q 8868-334-56    T 8478  ☑

40500/40500 [==============================] - 14s 344us/step - loss: 0.0163 - acc: 0.9949 - val_loss: 0.5945 - val_acc: 0.8769
Q 798-638+2      T 162   ☑ 162  
Q 6188-554-5     T 5629  ☑ 5629 
Q 8777+323-14    T 9086  ☑ 9086 
Q 4341-3-1       T 4337  ☑ 4337 
Q 4804-273-12    T 4519  ☒ 4518 
Q 4535-1710-64   T 2761  ☒ 2751 
Q 2680+9-6       T 2683  ☑ 2683 
Q 2843+2192+584  T 5619  ☒ 5576 
Q 3259-673+32    T 2618  ☒ 2609 
Q 7001+1758+918  T 9677  ☒ 17878

--------------------------------------------------
Iteration 91
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/step - loss: 0.0262 - acc: 0.9916 - val_loss: 0.5889 - val_acc: 0.8768
Q 652+579+1      T 1232  ☑ 1232 
Q 5276+2487+30   T 7793  ☒ 7892 
Q 246+61+21      T 328   ☑ 328  
Q 7725-797+284   T 7212  ☒ 7210 
Q 399-339+91     T 151   ☒ 14   
Q 1840+464-4     T 2300  ☒ 2290 
Q 354-31-7       T 316   ☑ 316  
Q 2932+909+22    T 3863  ☒ 3864 
Q 630-553-70     T 7     ☒

### Analysis
* addition & subtraction
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* Best accuracy: 89.89%

# Multiplication

In [17]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + DIGITS
LAYERS = 3
chars = '0123456789* '
DATA_SIZE = 80000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('*', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 80000
Vectorization...
Build model...

--------------------------------------------------
Iteration 0
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 24s 443us/step - loss: 1.8753 - acc: 0.2928 - val_loss: 1.7975 - val_acc: 0.3039
Q 265*49  T 12985  ☒ 22400 
Q 307*91  T 27937  ☒ 12475 
Q 907*72  T 65304  ☒ 12148 
Q 952*733 T 697816 ☒ 224455
Q 521*181 T 94301  ☒ 10453 
Q 155*85  T 13175  ☒ 1255  
Q 470*60  T 28200  ☒ 16600 
Q 417*185 T 77145  ☒ 11155 
Q 548*174 T 95352  ☒ 111468
Q 921*374 T 344454 ☒ 224558

--------------------------------------------------
Iteration 1
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 365us/step - loss: 1.5905 - acc: 0.3767 - val_loss: 1.5471 - val_acc: 0.3839
Q 870*365 T 317550 ☒ 387750
Q 819*45  T 36855  ☒ 38875 
Q 650*647 T 420550 ☒ 488750
Q 663*46  T 30498  ☒ 38806 
Q 182*57  T 10374  ☒ 10

54000/54000 [==============================] - 20s 363us/step - loss: 0.6523 - acc: 0.7457 - val_loss: 0.7984 - val_acc: 0.6813
Q 832*134 T 111488 ☒ 113728
Q 842*389 T 327538 ☒ 327818
Q 516*268 T 138288 ☒ 141248
Q 887*70  T 62090  ☑ 62090 
Q 225*84  T 18900  ☒ 18300 
Q 695*563 T 391285 ☒ 396685
Q 800*78  T 62400  ☒ 62600 
Q 889*766 T 680974 ☒ 673854
Q 464*346 T 160544 ☒ 169604
Q 770*53  T 40810  ☒ 41010 

--------------------------------------------------
Iteration 14
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 364us/step - loss: 0.6280 - acc: 0.7577 - val_loss: 0.7857 - val_acc: 0.6873
Q 548*34  T 18632  ☒ 18192 
Q 124*75  T 9300   ☑ 9300  
Q 248*94  T 23312  ☒ 23912 
Q 886*762 T 675132 ☒ 670912
Q 959*56  T 53704  ☒ 53204 
Q 100*60  T 6000   ☑ 6000  
Q 432*88  T 38016  ☒ 37416 
Q 674*82  T 55268  ☒ 54708 
Q 639*237 T 151443 ☒ 146083
Q 418*23  T 9614   ☒ 9434  

--------------------------------------------------
Iteratio

Q 856*39  T 33384  ☒ 33404 
Q 619*74  T 45806  ☒ 45006 
Q 418*63  T 26334  ☒ 26674 
Q 804*764 T 614256 ☒ 611196
Q 830*67  T 55610  ☒ 55410 
Q 678*258 T 174924 ☒ 178524
Q 179*72  T 12888  ☒ 12188 
Q 396*64  T 25344  ☒ 25404 
Q 657*469 T 308133 ☒ 316413
Q 534*490 T 261660 ☒ 262660

--------------------------------------------------
Iteration 27
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 362us/step - loss: 0.2211 - acc: 0.9323 - val_loss: 0.9252 - val_acc: 0.7282
Q 437*47  T 20539  ☒ 20339 
Q 749*52  T 38948  ☒ 38648 
Q 233*44  T 10252  ☑ 10252 
Q 871*359 T 312689 ☒ 306529
Q 906*288 T 260928 ☒ 261128
Q 955*63  T 60165  ☒ 60665 
Q 928*21  T 19488  ☒ 19448 
Q 649*86  T 55814  ☒ 55134 
Q 940*837 T 786780 ☒ 783320
Q 755*30  T 22650  ☑ 22650 

--------------------------------------------------
Iteration 28
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 363us/step - 

54000/54000 [==============================] - 20s 362us/step - loss: 0.0411 - acc: 0.9919 - val_loss: 1.3746 - val_acc: 0.7252
Q 129*18  T 2322   ☑ 2322  
Q 737*170 T 125290 ☒ 124290
Q 944*88  T 83072  ☒ 83332 
Q 746*10  T 7460   ☑ 7460  
Q 882*49  T 43218  ☒ 43238 
Q 581*418 T 242858 ☒ 246058
Q 537*103 T 55311  ☒ 54911 
Q 119*60  T 7140   ☑ 7140  
Q 828*97  T 80316  ☑ 80316 
Q 165*89  T 14685  ☒ 14985 

--------------------------------------------------
Iteration 41
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 362us/step - loss: 0.0710 - acc: 0.9808 - val_loss: 1.3861 - val_acc: 0.7185
Q 863*613 T 529019 ☒ 538179
Q 774*19  T 14706  ☒ 14586 
Q 568*139 T 78952  ☒ 89112 
Q 848*221 T 187408 ☒ 188548
Q 120*68  T 8160   ☒ 8140  
Q 168*55  T 9240   ☒ 9120  
Q 582*34  T 19788  ☒ 19108 
Q 933*10  T 9330   ☑ 9330  
Q 554*413 T 228802 ☒ 227502
Q 771*199 T 153429 ☒ 157429

--------------------------------------------------
Iteratio

Q 221*78  T 17238  ☒ 17038 
Q 650*44  T 28600  ☑ 28600 
Q 591*162 T 95742  ☒ 95902 
Q 826*151 T 124726 ☒ 124226
Q 416*332 T 138112 ☒ 146572
Q 392*139 T 54488  ☒ 52168 
Q 625*402 T 251250 ☒ 250450
Q 882*99  T 87318  ☑ 87318 
Q 730*175 T 127750 ☒ 124750
Q 156*94  T 14664  ☑ 14664 

--------------------------------------------------
Iteration 54
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 363us/step - loss: 0.0212 - acc: 0.9955 - val_loss: 1.6045 - val_acc: 0.7246
Q 567*167 T 94689  ☒ 84289 
Q 807*31  T 25017  ☑ 25017 
Q 626*462 T 289212 ☒ 284952
Q 450*450 T 202500 ☒ 102500
Q 200*47  T 9400   ☒ 9100  
Q 217*90  T 19530  ☑ 19530 
Q 472*44  T 20768  ☑ 20768 
Q 529*35  T 18515  ☒ 18015 
Q 824*91  T 74984  ☒ 74904 
Q 124*41  T 5084   ☒ 5164  

--------------------------------------------------
Iteration 55
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 362us/step - 

54000/54000 [==============================] - 20s 362us/step - loss: 0.0211 - acc: 0.9948 - val_loss: 1.7163 - val_acc: 0.7268
Q 688*92  T 63296  ☒ 63496 
Q 913*41  T 37433  ☒ 37453 
Q 741*670 T 496470 ☒ 493670
Q 942*30  T 28260  ☑ 28260 
Q 752*111 T 83472  ☒ 82712 
Q 406*87  T 35322  ☒ 35822 
Q 338*22  T 7436   ☒ 7636  
Q 867*682 T 591294 ☒ 592274
Q 587*95  T 55765  ☑ 55765 
Q 848*145 T 122960 ☒ 124660

--------------------------------------------------
Iteration 68
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 363us/step - loss: 0.0290 - acc: 0.9919 - val_loss: 1.7090 - val_acc: 0.7273
Q 377*202 T 76154  ☒ 76854 
Q 648*416 T 269568 ☒ 268848
Q 215*39  T 8385   ☒ 8265  
Q 672*52  T 34944  ☒ 34344 
Q 971*476 T 462196 ☒ 453596
Q 250*63  T 15750  ☒ 15450 
Q 679*552 T 374808 ☒ 376108
Q 933*908 T 847164 ☒ 852004
Q 358*13  T 4654   ☑ 4654  
Q 823*164 T 134972 ☒ 138812

--------------------------------------------------
Iteratio

Q 679*97  T 65863  ☒ 65563 
Q 686*319 T 218834 ☒ 216754
Q 932*760 T 708320 ☒ 701520
Q 364*88  T 32032  ☒ 32452 
Q 740*22  T 16280  ☑ 16280 
Q 913*790 T 721270 ☒ 711470
Q 826*81  T 66906  ☑ 66906 
Q 731*45  T 32895  ☑ 32895 
Q 786*91  T 71526  ☒ 71726 
Q 214*83  T 17762  ☒ 17662 

--------------------------------------------------
Iteration 81
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 364us/step - loss: 0.0098 - acc: 0.9978 - val_loss: 1.8004 - val_acc: 0.7317
Q 923*100 T 92300  ☑ 92300 
Q 247*92  T 22724  ☒ 22924 
Q 485*193 T 93605  ☒ 90900 
Q 449*157 T 70493  ☒ 74573 
Q 819*757 T 619983 ☒ 624023
Q 305*37  T 11285  ☒ 11385 
Q 219*10  T 2190   ☑ 2190  
Q 258*86  T 22188  ☒ 22888 
Q 475*131 T 62225  ☒ 62425 
Q 816*517 T 421872 ☒ 428372

--------------------------------------------------
Iteration 82
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 365us/step - 

54000/54000 [==============================] - 20s 363us/step - loss: 0.0251 - acc: 0.9926 - val_loss: 1.9074 - val_acc: 0.7222
Q 473*27  T 12771  ☒ 12591 
Q 179*43  T 7697   ☒ 7417  
Q 258*258 T 66564  ☒ 66704 
Q 401*34  T 13634  ☑ 13634 
Q 840*12  T 10080  ☒ 10280 
Q 786*774 T 608364 ☒ 616764
Q 940*463 T 435220 ☒ 434220
Q 404*187 T 75548  ☒ 70548 
Q 982*85  T 83470  ☒ 83370 
Q 226*71  T 16046  ☒ 16106 

--------------------------------------------------
Iteration 95
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 20s 363us/step - loss: 0.0269 - acc: 0.9922 - val_loss: 1.8646 - val_acc: 0.7307
Q 841*423 T 355743 ☒ 353843
Q 647*47  T 30409  ☒ 30109 
Q 547*84  T 45948  ☒ 45128 
Q 577*81  T 46737  ☒ 47337 
Q 600*54  T 32400  ☑ 32400 
Q 854*416 T 355264 ☒ 356124
Q 257*39  T 10023  ☒ 9903  
Q 760*51  T 38760  ☑ 38760 
Q 886*464 T 411104 ☒ 417604
Q 434*15  T 6510   ☑ 6510  

--------------------------------------------------
Iteratio

### Analysis
* Multiplication
* 3 digits
* Need to modify the maximum length of answer
* Training size: 54K
* Total training epoch: 100 epoch
* Best accuracy: 73.48%

# Testing

In [18]:
print("MSG : Prediction")
for i in range(10):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([ind])], test_y[np.array([ind])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

MSG : Prediction
Q 125*9   T 1125   ☑ 1125  
Q 509*6   T 3054   ☑ 3054  
Q 987*283 T 279321 ☒ 281901
Q 819*73  T 59787  ☒ 69987 
Q 369*88  T 32472  ☒ 32072 
Q 973*55  T 53515  ☒ 53315 
Q 322*5   T 1610   ☑ 1610  
Q 93*3    T 279    ☒ 2790  
Q 770*111 T 85470  ☒ 85270 
Q 824*18  T 14832  ☒ 14812 


# Report 
* Addition only or substraction only both can reach 99% evaluate accuracy
* Combine addition+substraction both can almost reach 90% evaluate accuracy
  (3 digits: 96.52% 4digits:89.89%)
* The more digits need to be computed, the more training data need to make accuracy higher
* Multiplication is more difficult than addition+substraction. Even use 54K training data to train, just reach almost 75% validation accuracy
* According to [Learning to Execute](http://arxiv.org/abs/1410.4615), reverse input can get higher accuracy